In [1]:
import torch
from torch import nn
import random,numpy
import numpy as np
from copy import deepcopy
from game import Game, Move, RandomPlayer, MyPlayer,translate_number_to_position_direction,translate_number_to_position,TrainedPlayer
import torch.nn.init as init

## Training the agent

In [2]:
## Constants definition

# These parameter have been obtained by running a genetic algorithm over individuals representing the network hyper parameters
# Based on the fact that the agent could start as first/second, we might have different hyperparameters for the network. The 
# following, represents a possible base to start from (model version for second turn)

num_iterations = 800
num_matches = 6
max_dim_replay_buff = 589
time_to_update = 27
gamma = 0.280
batch_size= 409

device = (
    "cuda:0"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cpu device


In [3]:
class ReplayBuffer:    
    def __init__(self, buffer_size):
        self.buffer_size = buffer_size        
        self.buffer = []
        self.position = 0
        
    def add_experience(self, experience):        
        if len(self.buffer) < self.buffer_size:
            self.buffer.append(experience)        
        else:
            self.buffer[self.position] = experience        
            self.position = (self.position + 1) % self.buffer_size
            
    def sample_batch(self, batch_size):
        batch_indices = np.random.choice(len(self.buffer), batch_size, replace=True)
        batch = [self.buffer[i] for i in batch_indices]
        return batch

In [4]:
from game import Player
from tqdm import tqdm


def training(agent_to_train: 'Player', opponent: 'Player', starts_first: bool):
    loss_tot = []
    taboo_set = set()

    replay_buff = ReplayBuffer(max_dim_replay_buff) # replay buffer, from which we sample for BATCH learning
    torch.set_grad_enabled(True)
    
    for step in tqdm(range(num_iterations), desc="Training Iterations"):  
        # we fill the replay buffer with experiences made with matches
        for match in range(num_matches):
            winner = -1
            g = Game()
            g.current_player_idx = int(starts_first) # If TRUE it is set to 1, there the agents starts as first 
            players = [agent_to_train, opponent] 
            go = False 

            while winner < 0:
                    g.current_player_idx += 1
                    g.current_player_idx %= len(players)
                    prev_state=deepcopy(g)
                    from_pos, slide = players[g.current_player_idx].make_move(g)
                    g._Game__move(from_pos, slide, g.current_player_idx)
                    
                    if g.current_player_idx==0:
                        reward=g.compute_reward()
                        go = True
                        
                    elif go and g.current_player_idx==1: 
                        if (tuple(prev_state.get_flat_board()), agent_to_train.last_action_number, tuple(deepcopy(g).get_flat_board())) not in taboo_set :
                            replay_row=(prev_state.get_flat_board(), agent_to_train.last_action_number, deepcopy(g), reward, reward==1)
                            taboo_set.add((tuple(prev_state.get_flat_board()),agent_to_train.last_action_number, tuple(deepcopy(g).get_flat_board())))
                            replay_buff.add_experience(replay_row)

                    if g.check_winner() != -1:
                        break
            
        batch_to_train = replay_buff.sample_batch(batch_size)
      
        #divide the batch
        state_batch, action_num, next_state_batch, reward, done = zip(*batch_to_train) 

        #forward the Generator
        q_values = agent_to_train.GeneratorNet(torch.tensor(state_batch, dtype=torch.float32)).to(agent_to_train.device)
        q_values_target = torch.zeros(batch_size, 44).to(agent_to_train.device)

        #update q_values target by using Bellman Equation  
        for i in range(batch_size): 
            if done[i] == False:
                q_values_target[i, action_num[i]] = reward[i] + gamma * torch.tensor(agent_to_train.compute_target(next_state_batch[i])).to(agent_to_train.device).item()
            else:
               q_values_target[i, action_num[i]] = reward[i] 
          
        agent_to_train.optimizer.zero_grad()
        loss_curr=agent_to_train.criterion(q_values,q_values_target).to(agent_to_train.device)
        loss_curr.backward()
    
        agent_to_train.optimizer.step()

        loss_tot.append(loss_curr)
      
        if (step % time_to_update) == 0:
            # update the parameter of the TargetNet
            agent_to_train.copy_params_TargetNet()
            
   
    return agent_to_train.GeneratorNet.state_dict(), loss_tot
            
      

In [5]:
agent_train_first = MyPlayer()
agent_train_second = MyPlayer()
opponent = RandomPlayer()

TrainedGeneratorNet_first = TrainedPlayer()
trained_model_params_first, loss_tot_first = training(agent_train_first, opponent, True) # this model is trained to start first
TrainedGeneratorNet_first.GeneratorNet.load_state_dict(trained_model_params_first) 


Training Iterations:   0%|          | 0/300 [00:00<?, ?it/s]C:\Users\chito\AppData\Local\Temp\ipykernel_4316\1211647654.py:47: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:264.)
  q_values = agent_to_train.GeneratorNet(torch.tensor(state_batch, dtype=torch.float32)).to(agent_to_train.device)
Training Iterations: 100%|██████████| 300/300 [00:50<00:00,  5.93it/s]


<All keys matched successfully>

In [6]:
"""
Change the following settings based on the genetic algorithm for the model version that start as second

num_iterations = ...
num_matches = ...
max_dim_replay_buff = ...
time_to_update = ...
gamma = ...
batch_size = ...
"""


TrainedGeneratorNet_second = TrainedPlayer()
trained_model_params_second, loss_tot_second = training(agent_train_second, opponent, False) # this model is trained to start second
TrainedGeneratorNet_second.GeneratorNet.load_state_dict(trained_model_params_second)

Training Iterations:  17%|█▋        | 52/300 [00:08<00:42,  5.90it/s]

## Loss function visualization

In [ ]:
from matplotlib import pyplot as plt

vals_first = [el.detach().numpy() for el in loss_tot_first]
vals_second = [el.detach().numpy() for el in loss_tot_second]

# Generate a sequence of integers to represent the epoch numbers
epochs = range(1, len(loss_tot_second) + 1)

# Plot and label the training and validation loss values
plt.plot(epochs, vals_first, label='Training Loss (first turn)')
plt.plot(epochs, vals_second, label='Training Loss (second turn)')


# Add title and axes labels
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

# Display the plot
plt.legend()
plt.show()

In [ ]:
torch.save(TrainedGeneratorNet_first.GeneratorNet.state_dict(), './trained_models/first_turn_model.pth')
torch.save(TrainedGeneratorNet_second.GeneratorNet.state_dict(), './trained_models/second_turn_model.pth')

In [ ]:
from game import Game, RandomPlayer, MyPlayer, TrainedPlayer, TrainedPlayer_Complete
from tqdm import tqdm
import torch

TrainedGeneratorNet = TrainedPlayer()
TrainedGeneratorNet.GeneratorNet.load_state_dict(torch.load('./trained_models/try_second.pth'))

wins = 0
losts = 0
draws = 0
num_match_test = 100

for step in tqdm(range(num_match_test), desc="Game"):
    player=RandomPlayer()
    g=Game()
    #g.current_player_idx = step%2
    #g.current_player_idx = 1
    winner=g.play(TrainedGeneratorNet, player)
    if winner==0:
        wins+=1
       
    if winner==1:
        losts+=1
        
    if winner ==-1:
        draws+=1

print(f"Accuracy: {(wins/num_match_test)*100}")